In [1]:
import pandas as pd
import altair as alt

In [2]:
auftraggeber = pd.read_csv('auftraggeber.csv')

In [36]:
auftrag_per = alt.Chart(auftraggeber.groupby('auftraggeber-per-name').count().reset_index()).mark_bar().encode(
        alt.X('auftraggeber-per-name:N', sort='-y', title='Auftraggeber'),
        alt.Y('title:Q', title='Datensätze'),
        alt.Color('auftraggeber-per-name:N', title='Auftraggeber', sort='-y'),
        tooltip=[alt.Tooltip('auftraggeber-per-name:N', title='Auftraggeber'), alt.Tooltip('title:Q', title='Anzahl')]
    ).transform_window(
    rank='rank(title)',
    sort=[alt.SortField('title', order='descending')]
    ).transform_filter(
    (alt.datum.rank < 20))

In [37]:
auftrag_per

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [38]:
auftrag_inst = alt.Chart(auftraggeber.groupby('auftraggeber-inst-name').count().reset_index()).mark_bar().encode(
        alt.X('auftraggeber-inst-name:N', sort='-y', title='Auftraggeber'),
        alt.Y('title:Q', title='Datensätze'),
        alt.Color('auftraggeber-inst-name:N', sort='-y', title='institutionelle Auftraggeber'),
        tooltip=[alt.Tooltip('auftraggeber-inst-name:N', title='Auftraggeber'), alt.Tooltip('title:Q', title='Anzahl Datensätze')]
    ).transform_window(
    rank='rank(title)',
    sort=[alt.SortField('title', order='descending')]
    ).transform_filter(
    (alt.datum.rank < 20))

In [39]:
auftrag_inst

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [46]:
entstehungsorte = pd.read_csv('entstehungsorte.csv')
orte = pd.read_csv('orte-geo.csv')

In [48]:
import geopy.geocoders as geo
from geopy.extra.rate_limiter import RateLimiter
geolocator = geo.Nominatim(user_agent='rundschreiben')
geo.options.default_timeout = 10
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=.5)


In [49]:
orte['location'] = orte['ort-name'].apply(geocode)
orte['lat'] = orte['location'].apply(lambda loc: loc.latitude if loc else None)
orte['lon'] = orte['location'].apply(lambda loc: loc.longitude if loc else None)

In [51]:
orte.to_csv('orte-geo-koord.csv', index=None)

In [53]:
entstehungsorte = entstehungsorte.merge(orte, on='ort-name')

In [68]:
ort_anzahl = pd.DataFrame(entstehungsorte.value_counts(['ort-name']).reset_index())
ort_anzahl.columns = ['ort-name','n']

In [72]:
orte_fin = orte.merge(ort_anzahl, on='ort-name')

In [117]:
import pydeck as pdk
#Karte
INITIAL_VIEW_STATE = pdk.ViewState(
    latitude=50.67877877706058,
    longitude=8.129981238464392,
    zoom=4.5,
    max_zoom=16,
    bearing=0,
    )

scatterplotlayer = pdk.Layer(
    "ScatterplotLayer",
    orte_fin[['ort-name','lat','lon','n']],
    pickable=True,
    opacity=0.5,
    stroked=True,
    filled=True,
    radius_min_pixels=2,
    line_width_min_pixels=1,
    get_position='[lon, lat]',
    get_radius="n*50",
    get_fill_color=[255, 140, 0],
    get_line_color=[0, 0, 0],
)

r = pdk.Deck(
    scatterplotlayer,
    initial_view_state=INITIAL_VIEW_STATE,
    map_style=pdk.map_styles.LIGHT
)

In [109]:
r.to_html()